In [77]:
import itertools
nano_raw = [100.2,101.3,101.2,107.3,103.4,100.6,100.7,108.2,115,107.6,129.8,135,134.6,500,549,558,12,13,14,18,15]

In [78]:
nanoen = list(enumerate(nano_raw))
print(nanoen)

[(0, 100.2), (1, 101.3), (2, 101.2), (3, 107.3), (4, 103.4), (5, 100.6), (6, 100.7), (7, 108.2), (8, 115), (9, 107.6), (10, 129.8), (11, 135), (12, 134.6), (13, 500), (14, 549), (15, 558), (16, 12), (17, 13), (18, 14), (19, 18), (20, 15)]


In [83]:
nanoavg = []
index_map = {p:nano for p, nano in nanoen}
skip_until = 0
for p, nano in nanoen:
    if p < skip_until:
        continue
    if index_map[p] > 299:
        if int(p + 1) in index_map.keys():
            if abs(index_map[p] - index_map[p+1]) <= 10:
                nanoavg.append((round((index_map[p]+index_map[p+1])/2,1)))
                skip_until = p + 2
            else:
                combos = itertools.combinations([p,p+1,p+2,p+3,p+4,p+5,p+6,p+7,p+8],2)
                for p1,p2 in combos:
                    if p1 and p2 in index_map.keys():
                        if (abs(index_map[p1]-index_map[p2])) <= 10:
                            nanoavg.append((round((index_map[p1]+index_map[p2])/2,1)))
                            if p1 > p2:
                                skip_until = p1 + 1
                                break
                            elif p2 > p1:
                                skip_until = p2 + 1
                                break
    if index_map[p] < 99:
        if int(p + 1) in index_map.keys():
            if abs(index_map[p] - index_map[p+1]) <= 2:
                nanoavg.append((round((index_map[p]+index_map[p+1])/2,1)))
                skip_until = p + 2
            else:
                combos = itertools.combinations([p,p+1,p+2,p+3,p+4,p+5,p+6,p+7,p+8],2)
                for p1,p2 in combos:
                    if p1 and p2 in index_map.keys():
                        if (abs(index_map[p1]-index_map[p2])) <= 2:
                            nanoavg.append((round((index_map[p1]+index_map[p2])/2,1)))
                            if p1 > p2:
                                skip_until = p1 + 1
                                break
                            elif p2 > p1:
                                skip_until = p2 + 1
                                break
    if index_map[p] > 99:
        if int(p + 1) in index_map.keys():
            if abs(index_map[p] - index_map[p+1]) <= 5:
                nanoavg.append((round((index_map[p]+index_map[p+1])/2,1)))
                skip_until = p + 2
            else:
                combos = itertools.combinations([p,p+1,p+2,p+3,p+4,p+5,p+6,p+7,p+8],2)
                for p1,p2 in combos:
                    if p1 and p2 in index_map.keys():
                        if (abs(index_map[p1]-index_map[p2])) <= 5:
                            nanoavg.append((round((index_map[p1]+index_map[p2])/2,1)))
                            if p1 > p2:
                                skip_until = p1 + 1
                                break
                            elif p2 > p1:
                                skip_until = p2 + 1
                                break
    else:
        break
print(nanoavg)

[100.8, 102.3, 100.7, 107.9, 132.2, 553.5, 12.5, 14.5]
